In [1]:
import json
import os
import time

import get_youtube8m_features as yt_features
import frame_level_models
import video_level_models
import readers
import utils
import eval_util
import export_model
import losses

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow import app
from tensorflow import gfile
from tensorflow import logging
from tensorflow.python.client import device_lib

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Maak alle flags aan

# Dataset flags
flags.DEFINE_string("train_dir", "models",
                  "The directory to save the model files in.")
flags.DEFINE_string(
  "train_data_pattern", "train_data/*.tfrecord",
  "File glob for the training dataset. If the files refer to Frame Level "
  "features (i.e. tensorflow.SequenceExample), then set --reader_type "
  "format. The (Sequence)Examples are expected to have 'rgb' byte array "
  "sequence feature as well as a 'labels' int64 context feature.")
flags.DEFINE_string("feature_names", "mean_rgb", "Name of the feature "
                  "to use for training.")
flags.DEFINE_string("feature_sizes", "1024", "Length of the feature vectors.")
flags.DEFINE_integer("num_classes", 37, "Number of classes in dataset")

In [3]:
# Model flags
flags.DEFINE_bool(
  "frame_features", False,
  "If set, then --train_data_pattern must be frame-level features. "
  "Otherwise, --train_data_pattern must be aggregated video-level "
  "features. The model must also be set appropriately (i.e. to read 3D "
  "batches VS 4D batches.")
flags.DEFINE_string(
  "model", "LogisticModel",
  "Which architecture to use for the model. Models are defined "
  "in models.py.")
flags.DEFINE_bool(
  "start_new_model", True,
  "If set, this will not resume from a checkpoint and will instead create a"
  " new model instance.")

In [4]:
# Training flags
flags.DEFINE_integer("batch_size", 1024,
                   "How many examples to process per batch for training.")
flags.DEFINE_string("label_loss", "CrossEntropyLoss",
                  "Which loss function to use for training the model.")
flags.DEFINE_float("regularization_penalty", 1.0,
                  "How much weight to give to the regularization loss (the label loss has "
                  "a weight of 1).")
flags.DEFINE_float("base_learning_rate", 0.01,
                 "Which learning rate to start with.")
flags.DEFINE_float("learning_rate_decay", 0.95,
                 "Learning rate decay factor to be applied every "
                 "learning_rate_decay_examples.")
flags.DEFINE_float("learning_rate_decay_examples", 4000000,
                 "Multiply current learning rate by learning_rate_decay "
                 "every learning_rate_decay_examples.")
flags.DEFINE_integer("num_epochs", 10,
                   "How many passes to make over the dataset before "
                   "halting training.")
flags.DEFINE_integer("max_steps", None,
                   "The maximum number of iterations of the training loop.")
flags.DEFINE_integer("export_model_steps", 1000,
                   "The period, in number of steps, with which the model "
                   "is exported for batch prediction.")

In [5]:
# Other flags
flags.DEFINE_integer("num_readers", 8,
                   "How many threads to use for reading input files.")
flags.DEFINE_string("optimizer", "AdamOptimizer",
                  "What optimizer class to use.")
flags.DEFINE_float("clip_gradient_norm", 1.0, "Norm to clip gradients to.")
flags.DEFINE_bool(
  "log_device_placement", False,
  "Whether to write the device on which every op will run into the "
  "logs on startup.")

In [6]:
# Go to train data folder 
if os.getcwd().split('\\')[-1]!='train_data':
    os.chdir("./train_data")

## Download 1/1000th of the data (if not downloaded yet)
# yt_features.dwnld('2/video/train', 'eu', '1,100')
#yt_features.dwnld('1/video_level/train', 'us', '1,100')
os.chdir("..")

In [7]:
# Load the environment.
env = json.loads(os.environ.get("TF_CONFIG", "{}"))

In [8]:
# Load the cluster data from the environment.
cluster_data = env.get("cluster", None)
cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None

In [9]:
# Load the task data from the environment.
task_data = env.get("task", None) or {"type": "master", "index": 0}
task = type("TaskSpec", (object,), task_data)

In [10]:
def task_as_string(task):
  return "/job:%s/task:%s" % (task.type, task.index)

# Logging the version.
logging.set_verbosity(tf.logging.INFO)
logging.info("%s: Tensorflow version: %s.",
           task_as_string(task), tf.__version__)

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.8.0.


In [11]:
## Functie die elke keer wordt aangeroepen
def find_class_by_name(name, modules):
  """Searches the provided modules for the named class and returns it."""
  modules = [getattr(module, name, None) for module in modules]
  return next(a for a in modules if a)

In [12]:
## Define model
model = find_class_by_name(FLAGS.model,
    [frame_level_models, video_level_models])()

In [13]:
# Convert feature_names and feature_sizes to lists of values.
feature_names, feature_sizes = utils.GetListOfFeatureNamesAndSizes(
    FLAGS.feature_names, FLAGS.feature_sizes)
num_classes = FLAGS.num_classes

# Create reader
if FLAGS.frame_features:
    reader = readers.YT8MFrameFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)
else:
    reader = readers.YT8MAggregatedFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)

In [14]:
model_exporter = export_model.ModelExporter(
    frame_features=FLAGS.frame_features,
    model=model,
    reader=reader)

In [15]:
is_master = (task.type == "master" and task.index == 0)
config = tf.ConfigProto(
    allow_soft_placement=True,log_device_placement=FLAGS.log_device_placement)
max_steps_reached = False
last_model_export_step = 0

In [16]:
# Remove old model, if starting new model
if is_master and FLAGS.start_new_model:
    try:
      logging.info(
          "%s: Removing existing train directory.",
          task_as_string(task))
      gfile.DeleteRecursively(FLAGS.train_dir)
    except:
      logging.error(
          "%s: Failed to delete directory " + FLAGS.train_dir +
          " when starting a new model. Please delete it manually and" +
          " try again.", task_as_string(task))

INFO:tensorflow:/job:master/task:0: Removing existing train directory.
ERROR:tensorflow:/job:master/task:0: Failed to delete directory models when starting a new model. Please delete it manually and try again.


In [17]:
"""Starts a server if the execution is distributed."""
if cluster:
    logging.info("%s: Starting trainer within cluster %s.",
               task_as_string(task), cluster.as_dict())
    server = start_server(cluster, task)
    target = server.target
    device_fn = tf.train.replica_device_setter(
        ps_device="/job:ps",
        worker_device="/job:%s/task:%d" % (task.type, task.index),
        cluster=cluster)
else:
    target = ""
    device_fn = ""

In [18]:
def get_meta_filename(start_new_model, train_dir, task):
    if start_new_model:
      logging.info("%s: Flag 'start_new_model' is set. Building a new model.",
                   task_as_string(task))
      return None

    latest_checkpoint = tf.train.latest_checkpoint(train_dir)
    if not latest_checkpoint:
      logging.info("%s: No checkpoint file found. Building a new model.",
                   task_as_string(task))
      return None

    meta_filename = latest_checkpoint + ".meta"
    if not gfile.Exists(meta_filename):
      logging.info("%s: No meta graph file found. Building a new model.",
                     task_as_string(task))
      return None
    else:
      return meta_filename

meta_filename = get_meta_filename(FLAGS.start_new_model, FLAGS.train_dir, task)

INFO:tensorflow:/job:master/task:0: Flag 'start_new_model' is set. Building a new model.


In [28]:
## Functie die wordt aangeroepen door build_graph

def get_input_data_tensors(reader,
                           data_pattern,
                           batch_size=1000,
                           num_epochs=None,
                           num_readers=1):
  """Creates the section of the graph which reads the training data.

  Args:
    reader: A class which parses the training data.
    data_pattern: A 'glob' style path to the data files.
    batch_size: How many examples to process at a time.
    num_epochs: How many passes to make over the training data. Set to 'None'
                to run indefinitely.
    num_readers: How many I/O threads to use.

  Returns:
    A tuple containing the features tensor, labels tensor, and optionally a
    tensor containing the number of frames per video. The exact dimensions
    depend on the reader being used.

  Raises:
    IOError: If no files matching the given pattern were found.
  """
  logging.info("Using batch size of " + str(batch_size) + " for training.")
  with tf.name_scope("train_input"):
    files = gfile.Glob(data_pattern)
    if not files:
      raise IOError("Unable to find training files. data_pattern='" +
                    data_pattern + "'.")
    logging.info("Number of training files: %s.", str(len(files)))
    filename_queue = tf.train.string_input_producer(
        files, num_epochs=num_epochs, shuffle=True)
    training_data = [
        reader.prepare_reader(filename_queue) for _ in range(num_readers)
    ]

    print(training_data)
    
    ## Create batches by randomly shuffling tensors
    return tf.train.shuffle_batch_join(
        training_data,
        batch_size=batch_size,
        capacity=batch_size * 5,
        min_after_dequeue=batch_size,
        allow_smaller_final_batch=True,
        enqueue_many=True)

In [29]:
## Deze functie wordt aangeroepen om de Tensorflow-graph te bouwen

def build_graph(reader,
                model,
                train_data_pattern,
                label_loss_fn=losses.CrossEntropyLoss(),
                batch_size=1000,
                base_learning_rate=0.01,
                learning_rate_decay_examples=1000000,
                learning_rate_decay=0.95,
                optimizer_class=tf.train.AdamOptimizer,
                clip_gradient_norm=1.0,
                regularization_penalty=1,
                num_readers=1,
                num_epochs=None):
  """Creates the Tensorflow graph.

  This will only be called once in the life of
  a training model, because after the graph is created the model will be
  restored from a meta graph file rather than being recreated.

  Args:
    reader: The data file reader. It should inherit from BaseReader.
    model: The core model (e.g. logistic or neural net). It should inherit
           from BaseModel.
    train_data_pattern: glob path to the training data files.
    label_loss_fn: What kind of loss to apply to the model. It should inherit
                from BaseLoss.
    batch_size: How many examples to process at a time.
    base_learning_rate: What learning rate to initialize the optimizer with.
    optimizer_class: Which optimization algorithm to use.
    clip_gradient_norm: Magnitude of the gradient to clip to.
    regularization_penalty: How much weight to give the regularization loss
                            compared to the label loss.
    num_readers: How many threads to use for I/O operations.
    num_epochs: How many passes to make over the data. 'None' means an
                unlimited number of passes.
  """

  global_step = tf.Variable(0, trainable=False, name="global_step")
  print(global_step)
  local_device_protos = device_lib.list_local_devices()
  gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
  num_gpus = len(gpus)
  logging.info("Number of GPUs: %s.", str(num_gpus))

  if num_gpus > 0:
    logging.info("Using the following GPUs to train: " + str(gpus))
    num_towers = num_gpus
    device_string = '/gpu:%d'
  else:
    logging.info("No GPUs found. Training on CPU.")
    num_towers = 1
    device_string = '/cpu:%d'

  learning_rate = tf.train.exponential_decay(
      base_learning_rate,
      global_step * batch_size * num_towers,
      learning_rate_decay_examples,
      learning_rate_decay,
      staircase=True)
  tf.summary.scalar('learning_rate', learning_rate)

  optimizer = optimizer_class(learning_rate)

  ## Read input files
  unused_video_id, model_input_raw, labels_batch, num_frames = (
      get_input_data_tensors(
          reader,
          train_data_pattern,
          batch_size=batch_size * num_towers,
          num_readers=num_readers,
          num_epochs=num_epochs))
  tf.summary.histogram("model/input_raw", model_input_raw)
 
  feature_dim = len(model_input_raw.get_shape()) - 1

  model_input = tf.nn.l2_normalize(model_input_raw, feature_dim)

  tower_inputs = tf.split(model_input, num_towers)
  tower_labels = tf.split(labels_batch, num_towers)
  tower_num_frames = tf.split(num_frames, num_towers)
  tower_gradients = []
  tower_predictions = []
  tower_label_losses = []
  tower_reg_losses = []
  for i in range(num_towers):
    # For some reason these 'with' statements can't be combined onto the same
    # line. They have to be nested.
    with tf.device(device_string % i):
      with (tf.variable_scope(("tower"), reuse=True if i > 0 else None)):
        with (slim.arg_scope([slim.model_variable, slim.variable], device="/cpu:0" if num_gpus!=1 else "/gpu:0")):
          result = model.create_model(
            tower_inputs[i],
            num_frames=tower_num_frames[i],
            vocab_size=reader.num_classes,
            labels=tower_labels[i])
          #print(tower_labels[i])
          for variable in slim.get_model_variables():
            tf.summary.histogram(variable.op.name, variable)

          predictions = result["predictions"]
          tower_predictions.append(predictions)

          if "loss" in result.keys():
            label_loss = result["loss"]
          else:
            label_loss = label_loss_fn.calculate_loss(predictions, tower_labels[i])

          if "regularization_loss" in result.keys():
            reg_loss = result["regularization_loss"]
          else:
            reg_loss = tf.constant(0.0)

          reg_losses = tf.losses.get_regularization_losses()
          if reg_losses:
            reg_loss += tf.add_n(reg_losses)

          tower_reg_losses.append(reg_loss)

          # Adds update_ops (e.g., moving average updates in batch normalization) as
          # a dependency to the train_op.
          update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
          if "update_ops" in result.keys():
            update_ops += result["update_ops"]
          if update_ops:
            with tf.control_dependencies(update_ops):
              barrier = tf.no_op(name="gradient_barrier")
              with tf.control_dependencies([barrier]):
                label_loss = tf.identity(label_loss)

          tower_label_losses.append(label_loss)

          # Incorporate the L2 weight penalties etc.
          final_loss = regularization_penalty * reg_loss + label_loss
          gradients = optimizer.compute_gradients(final_loss,
              colocate_gradients_with_ops=False)
          tower_gradients.append(gradients)
  label_loss = tf.reduce_mean(tf.stack(tower_label_losses))
  tf.summary.scalar("label_loss", label_loss)
  if regularization_penalty != 0:
    reg_loss = tf.reduce_mean(tf.stack(tower_reg_losses))
    tf.summary.scalar("reg_loss", reg_loss)
  merged_gradients = utils.combine_gradients(tower_gradients)

  if clip_gradient_norm > 0:
    with tf.name_scope('clip_grads'):
      merged_gradients = utils.clip_gradient_norms(merged_gradients, clip_gradient_norm)

  train_op = optimizer.apply_gradients(merged_gradients, global_step=global_step)

  tf.add_to_collection("global_step", global_step)
  tf.add_to_collection("loss", label_loss)
  tf.add_to_collection("predictions", tf.concat(tower_predictions, 0))
  tf.add_to_collection("input_batch_raw", model_input_raw)
  tf.add_to_collection("input_batch", model_input)
  tf.add_to_collection("num_frames", num_frames)
  tf.add_to_collection("labels", tf.cast(labels_batch, tf.float32))
  tf.add_to_collection("train_op", train_op)

In [30]:
with tf.Graph().as_default() as graph:
    
    if meta_filename:
        logging.info("%s: Restoring from meta graph file %s",
            task_as_string(task), meta_filename)
        saver = tf.train.import_meta_graph(meta_filename)
    
    with tf.device(device_fn):

        if not meta_filename:
            """Find the model and build the graph."""

            label_loss_fn = find_class_by_name(FLAGS.label_loss, [losses])()
            optimizer_class = find_class_by_name(FLAGS.optimizer, [tf.train])

            build_graph(reader=reader,
                         model=model,
                         optimizer_class=optimizer_class,
                         clip_gradient_norm=FLAGS.clip_gradient_norm,
                         train_data_pattern=FLAGS.train_data_pattern,
                         label_loss_fn=label_loss_fn,
                         base_learning_rate=FLAGS.base_learning_rate,
                         learning_rate_decay=FLAGS.learning_rate_decay,
                         learning_rate_decay_examples=FLAGS.learning_rate_decay_examples,
                         regularization_penalty=FLAGS.regularization_penalty,
                         num_readers=FLAGS.num_readers,
                         batch_size=FLAGS.batch_size,
                         num_epochs=FLAGS.num_epochs)

        saver = tf.train.Saver(max_to_keep=0, keep_checkpoint_every_n_hours=0.25)
    
        global_step = tf.get_collection("global_step")[0]
        loss = tf.get_collection("loss")[0]
        predictions = tf.get_collection("predictions")[0]
        labels = tf.get_collection("labels")[0]
        train_op = tf.get_collection("train_op")[0]
        init_op = tf.global_variables_initializer()
        
    sv = tf.train.Supervisor(
        graph,
        logdir=FLAGS.train_dir,
        init_op=init_op,
        is_chief=is_master,
        global_step=global_step,
        save_model_secs=15 * 60,
        save_summaries_secs=120,
        saver=saver)

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>
INFO:tensorflow:Number of GPUs: 1.
INFO:tensorflow:Using the following GPUs to train: ['/device:GPU:0']
INFO:tensorflow:Using batch size of 1024 for training.
INFO:tensorflow:Number of training files: 41.
[(<tf.Tensor 'train_input/ParseExample/ParseExample:4' shape=(?,) dtype=string>, <tf.Tensor 'train_input/concat:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'train_input/SparseToIndicator:0' shape=(?, 4716) dtype=bool>, <tf.Tensor 'train_input/ones:0' shape=(?,) dtype=float32>), (<tf.Tensor 'train_input/ParseExample_1/ParseExample:4' shape=(?,) dtype=string>, <tf.Tensor 'train_input/concat_1:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'train_input/SparseToIndicator_1:0' shape=(?, 4716) dtype=bool>, <tf.Tensor 'train_input/ones_1:0' shape=(?,) dtype=float32>), (<tf.Tensor 'train_input/ParseExample_2/ParseExample:4' shape=(?,) dtype=string>, <tf.Tensor 'train_input/concat_2:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'train_inp

In [22]:
def export_model(global_step_val, saver, save_path, session):

    # If the model has already been exported at this step, return.
    if global_step_val == last_model_export_step:
      return

    last_checkpoint = saver.save(session, save_path, global_step_val)

    model_dir = "{0}/export/step_{1}".format(FLAGS.train_dir, global_step_val)
    logging.info("%s: Exporting the model at step %s to %s.",
                 task_as_string(task), global_step_val, model_dir)

    model_exporter.export_model(
        model_dir=model_dir,
        global_step_val=global_step_val,
        last_checkpoint=last_checkpoint)

In [23]:
config = tf.ConfigProto(
        allow_soft_placement=True,log_device_placement=FLAGS.log_device_placement)

In [24]:
logging.info("%s: Starting managed session.", task_as_string(task))

with sv.managed_session(target, config=config) as sess:
    
    try:
        logging.info("%s: Entering training loop.", task_as_string(task))
        while (not sv.should_stop()) and (not max_steps_reached):
            batch_start_time = time.time()
            _, global_step_val, loss_val, predictions_val, labels_val = sess.run(
                [train_op, global_step, loss, predictions, labels])
            seconds_per_batch = time.time() - batch_start_time
            examples_per_second = labels_val.shape[0] / seconds_per_batch

            if FLAGS.max_steps and FLAGS.max_steps <= global_step_val:
                max_steps_reached = True

            if is_master and global_step_val % 10 == 0 and FLAGS.train_dir:
                eval_start_time = time.time()
                hit_at_one = eval_util.calculate_hit_at_one(predictions_val, labels_val)
                perr = eval_util.calculate_precision_at_equal_recall_rate(predictions_val,
                                                                  labels_val)
                gap = eval_util.calculate_gap(predictions_val, labels_val)
                eval_end_time = time.time()
                eval_time = eval_end_time - eval_start_time

                logging.info("training step " + str(global_step_val) + " | Loss: " + ("%.2f" % loss_val) +
                             " Examples/sec: " + ("%.2f" % examples_per_second) + " | Hit@1: " +
                            ("%.2f" % hit_at_one) + " PERR: " + ("%.2f" % perr) +
                             " GAP: " + ("%.2f" % gap))

                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Hit@1", hit_at_one),
                    global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Perr", perr), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_GAP", gap), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("global_step/Examples/Second",
                                      examples_per_second), global_step_val)
                sv.summary_writer.flush()

                # Exporting the model every x steps
                time_to_export = ((last_model_export_step == 0) or
                    (global_step_val - last_model_export_step
                     >= FLAGS.export_model_steps))

                if is_master and time_to_export:
                    export_model(global_step_val, sv.saver, sv.save_path, sess)
                    last_model_export_step = global_step_val
            else:
                logging.info("training step " + str(global_step_val) + " | Loss: " +
                            ("%.2f" % loss_val) + " Examples/sec: " + ("%.2f" % examples_per_second))
    except tf.errors.OutOfRangeError:
        logging.info("%s: Done training -- epoch limit reached.")

logging.info("%s: Exited training loop.", task_as_string(task))
sv.Stop()

INFO:tensorflow:/job:master/task:0: Starting managed session.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path models\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:/job:master/task:0: Entering training loop.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:training step 1 | Loss: 3269.02 Examples/sec: 1807.95
INFO:tensorflow:models\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:training step 2 | Loss: 3243.53 Examples/sec: 8015.47
INFO:tensorflow:Recording summary at step 2.
INFO:tensorflow:training step 3 | Loss: 3217.62 Examples/sec: 8415.93
INFO:tensorflow:training step 4 | Loss: 3192.51 Examples/sec: 9333.88
INFO:tensorflow:training step 5 | Loss: 3164.59 Examples/sec: 10807.45
INFO:tensorflow:training step 6 | Loss: 3141.75 Examples/sec: 10695.31
INFO:tensorflow:training step 7 | Loss: 3114.57 Example

INFO:tensorflow:training step 99 | Loss: 1431.33 Examples/sec: 11672.12
INFO:tensorflow:training step 100 | Loss: 1418.40 Examples/sec: 10266.76 | Hit@1: 0.69 PERR: 0.52 GAP: 0.48
INFO:tensorflow:training step 101 | Loss: 1419.96 Examples/sec: 12590.15
INFO:tensorflow:training step 102 | Loss: 1393.36 Examples/sec: 9001.60
INFO:tensorflow:training step 103 | Loss: 1375.68 Examples/sec: 12330.73
INFO:tensorflow:training step 104 | Loss: 1369.42 Examples/sec: 10193.76
INFO:tensorflow:training step 105 | Loss: 1378.88 Examples/sec: 10319.63
INFO:tensorflow:training step 106 | Loss: 1332.38 Examples/sec: 10185.61
INFO:tensorflow:training step 107 | Loss: 1344.33 Examples/sec: 10282.72
INFO:tensorflow:training step 108 | Loss: 1333.60 Examples/sec: 9142.02
INFO:tensorflow:training step 109 | Loss: 1325.02 Examples/sec: 11529.19
INFO:tensorflow:training step 110 | Loss: 1312.75 Examples/sec: 9330.29 | Hit@1: 0.69 PERR: 0.53 GAP: 0.51
INFO:tensorflow:training step 111 | Loss: 1303.45 Examples

INFO:tensorflow:training step 208 | Loss: 540.62 Examples/sec: 9417.38
INFO:tensorflow:training step 209 | Loss: 530.45 Examples/sec: 10608.34
INFO:tensorflow:training step 210 | Loss: 530.12 Examples/sec: 9305.05 | Hit@1: 0.70 PERR: 0.54 GAP: 0.56
INFO:tensorflow:training step 211 | Loss: 518.89 Examples/sec: 9544.80
INFO:tensorflow:training step 212 | Loss: 509.64 Examples/sec: 10015.90
INFO:tensorflow:training step 213 | Loss: 516.47 Examples/sec: 9732.36
INFO:tensorflow:training step 214 | Loss: 513.02 Examples/sec: 9778.60
INFO:tensorflow:training step 215 | Loss: 510.18 Examples/sec: 9419.78
INFO:tensorflow:training step 216 | Loss: 495.41 Examples/sec: 10064.86
INFO:tensorflow:training step 217 | Loss: 496.26 Examples/sec: 10269.42
INFO:tensorflow:training step 218 | Loss: 486.41 Examples/sec: 9686.02
INFO:tensorflow:training step 219 | Loss: 489.20 Examples/sec: 10302.57
INFO:tensorflow:training step 220 | Loss: 480.88 Examples/sec: 10562.40 | Hit@1: 0.72 PERR: 0.54 GAP: 0.56
I

INFO:tensorflow:training step 318 | Loss: 189.65 Examples/sec: 11181.43
INFO:tensorflow:training step 319 | Loss: 188.70 Examples/sec: 8942.54
INFO:tensorflow:training step 320 | Loss: 189.78 Examples/sec: 10477.27 | Hit@1: 0.70 PERR: 0.53 GAP: 0.57
INFO:tensorflow:training step 321 | Loss: 184.94 Examples/sec: 11158.66
INFO:tensorflow:training step 322 | Loss: 183.03 Examples/sec: 11081.44
INFO:tensorflow:training step 323 | Loss: 182.49 Examples/sec: 10437.29
INFO:tensorflow:training step 324 | Loss: 181.01 Examples/sec: 9836.36
INFO:tensorflow:training step 325 | Loss: 177.71 Examples/sec: 10371.26
INFO:tensorflow:training step 326 | Loss: 177.53 Examples/sec: 10274.45
INFO:tensorflow:training step 327 | Loss: 174.55 Examples/sec: 10166.28
INFO:tensorflow:training step 328 | Loss: 173.14 Examples/sec: 8701.06
INFO:tensorflow:training step 329 | Loss: 172.22 Examples/sec: 8628.07
INFO:tensorflow:training step 330 | Loss: 169.09 Examples/sec: 9229.62 | Hit@1: 0.70 PERR: 0.54 GAP: 0.58

INFO:tensorflow:training step 429 | Loss: 69.52 Examples/sec: 5806.47
INFO:tensorflow:training step 430 | Loss: 69.30 Examples/sec: 6311.65 | Hit@1: 0.71 PERR: 0.52 GAP: 0.58
INFO:tensorflow:training step 431 | Loss: 68.85 Examples/sec: 10392.97
INFO:tensorflow:training step 432 | Loss: 66.81 Examples/sec: 4142.95
INFO:tensorflow:training step 433 | Loss: 67.95 Examples/sec: 9305.25
INFO:tensorflow:training step 434 | Loss: 66.22 Examples/sec: 6667.54
INFO:tensorflow:training step 435 | Loss: 64.96 Examples/sec: 7230.39
INFO:tensorflow:training step 436 | Loss: 65.91 Examples/sec: 8703.06
INFO:tensorflow:training step 437 | Loss: 65.62 Examples/sec: 5900.75
INFO:tensorflow:training step 438 | Loss: 65.20 Examples/sec: 7440.11
INFO:tensorflow:training step 439 | Loss: 63.89 Examples/sec: 5885.51
INFO:tensorflow:training step 440 | Loss: 63.07 Examples/sec: 7055.60 | Hit@1: 0.70 PERR: 0.53 GAP: 0.58
INFO:tensorflow:training step 441 | Loss: 62.78 Examples/sec: 9979.94
INFO:tensorflow:tra

In [27]:
with sv.managed_session(target, config=config) as sess:
    while (not sv.should_stop()) and (not max_steps_reached):
        batch_start_time = time.time()
        _, global_step_val, loss_val, predictions_val, labels_val = sess.run(
        [train_op, global_step, loss, predictions, labels])
        print(labels_val[0][0])
        #print(predictions_val[0][0])

INFO:tensorflow:Restoring parameters from models\model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path models\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 10.
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0